In [ ]:

#while (exponente := int(input("Dime un número entero y positivo : "))) not in range(0,1000001):  pass

    
  


In [16]:
#BUENO Captura!!!!!!
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import pandas as pd
import os
import base64

url = 'https://cndes-web.ses.mir.es/publico/Desaparecidos/'

# Configurar el driver de Selenium (asegúrate de tener ChromeDriver instalado)
driver = webdriver.Chrome()

# Agregar una pausa de 2 segundos después de cargar la página
time.sleep(2)

# Realizar la solicitud HTTP con Selenium
driver.get(url)

#Crear lista
desaparecidos=[]
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.btn.btn-info'))).click()
time.sleep(1)
for i in range(1,2):
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, f'/html/body/section[1]/div/div/div[{i}]/div/div[3]/div[2]/a'))).click()

    texto_columnas = driver.find_element(By.CLASS_NAME, 'fichaDet').text
    time.sleep(1)
    # Dividir el texto en líneas y procesar cada línea
    lineas = texto_columnas.split('\n')

    # Filtrar las líneas para excluir elementos no deseados
    lineas_filtradas = [linea for linea in lineas if not linea.startswith(('Compartir', 'INFORMAR'))]

    # Crear un diccionario
    info_dict = {'NOMBRE': lineas_filtradas[0]}

    # Recorrer las líneas desde la segunda posición y agregar al diccionario
    for i in range(1, len(lineas_filtradas)-1, 2):
        etiqueta = lineas_filtradas[i]
        valor = lineas_filtradas[i + 1]
        info_dict[etiqueta] = valor
        
    desaparecidos.append(info_dict)
    driver.back()
# Imprimir el diccionario
print(desaparecidos)
#HAY QUE CREAR CON ELEMENTO <div class="localizado">localizado</div> PARA ELIMINAR O CREAR OTRO CAMPO SI APARECE CON LOCALIZADO{ESTADO:LOCALIZADO}


[{'NOMBRE': 'VIKTORIIA KUDRIA', 'DESAPARECE': '30 de enero de 2024', 'FECHA NACIMIENTO': '02 de octubre de 1981', 'EDAD AL DESAPARECER': '42 años', 'EDAD ACTUAL': '42 años', 'LUGAR DE DESAPARICIÓN': 'ELCHE/ELX. ALICANTE/ALACANT, COMUNIDAD VALENCIANA'}]


In [20]:
#buena coge imagenes!!!!!


# Definir la lista de prueba
lista_prueba=[{'NOMBRE': 'VIKTORIIA KUDRIA', 'DESAPARECE': '30 de enero de 2024', 'FECHA NACIMIENTO': '02 de octubre de 1981', 'EDAD AL DESAPARECER': '42 años', 'EDAD ACTUAL': '42 años', 'LUGAR DE DESAPARICIÓN': 'ELCHE/ELX. ALICANTE/ALACANT, COMUNIDAD VALENCIANA'}]

url = 'https://cndes-web.ses.mir.es/publico/Desaparecidos/Buscar.html'

lista_fallos=[]
# Configurar el driver de Selenium (asegúrate de tener ChromeDriver instalado)
driver_2 = webdriver.Chrome()

# Agregar una pausa de 2 segundos después de cargar la página
time.sleep(2)

# Realizar la solicitud HTTP con Selenium
driver_2.get(url)
WebDriverWait(driver_2, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.btn.btn-info'))).click()
#WebDriverWait(driver_2, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'btn.btnFor.titulo.collapsed'))).click()
WebDriverWait(driver_2, 5).until(EC.element_to_be_clickable((By.ID, 'desp1'))).click()


# Esperar hasta que el elemento con el ID 'nombre' sea visible
#elemento_nombre = WebDriverWait(driver_2, 5).until(EC.visibility_of_element_located((By.ID, 'nombre')))
#elemento_nombre.send_keys(nombre)

#elemento_apellido = WebDriverWait(driver_2, 5).until(EC.visibility_of_element_located((By.ID, 'apellido2')))
#elemento_nombre.send_keys(apellido)

# Resto del código...
for i in lista_prueba:
    nombre = i['NOMBRE'].split(' ')[0]
    a = i['NOMBRE'].split(' ')
    apellido=a[len(a)-1]
    # Esperar hasta que el elemento con el ID 'nombre' sea visible
    elemento_nombre = WebDriverWait(driver_2, 5).until(EC.visibility_of_element_located((By.ID, 'nombre')))
    #elemento_nombre.clear()  # Limpiar el campo de nombre
    elemento_nombre.send_keys(nombre)

    # Esperar hasta que el elemento con el ID 'apellido1' sea visible
    elemento_apellido = WebDriverWait(driver_2, 5).until(EC.visibility_of_element_located((By.ID, 'apellido2')))
    #elemento_apellido.clear()  # Limpiar el campo de apellido
    elemento_apellido.send_keys(apellido)

    try:
        WebDriverWait(driver_2, 10).until(EC.element_to_be_clickable((By.ID, 'enviar'))).click()

        # Pausa para asegurar que la página haya cargado completamente
        time.sleep(4)

        # Obtener la URL de la imagen (reemplaza el xpath con el correcto)
        img_element = driver_2.find_element(By.XPATH, "/html/body/section[2]/div/div[2]/div/div/div[1]/a/img")
        img_data_base64 = img_element.get_attribute("src").split(",")[1]  # Extraer la parte de base64

        # Decodificar y guardar la imagen en la carpeta "imagenes"
        carpeta_imagenes = "imagenes"
        ruta_imagen = os.path.join(carpeta_imagenes, f"{i['NOMBRE']}.png")

        # Guardar la imagen en la carpeta "imagenes"
        os.makedirs(carpeta_imagenes, exist_ok=True)
        with open(ruta_imagen, "wb") as img_file:
            img_file.write(base64.b64decode(img_data_base64))

        # Añadir la ruta de la imagen al diccionario actual
        i['imagen'] = ruta_imagen

    except Exception as e:
        WebDriverWait(driver_2, 5).until(EC.element_to_be_clickable((By.ID, 'buttonAceptarError'))).click()
        print(f"Error al procesar {i['NOMBRE']}")
        #time.sleep(2)
        
        elemento_apellido.clear()  # Limpiar el campo de apellido
        elemento_nombre.clear()  # Limpiar el campo de nombre
        #driver_2.find_element(By.XPATH, "/html/body/header/div[1]/div/div[3]/div/div[2]/a").click()#boton de buscar
        #WebDriverWait(driver_2, 5).until(EC.element_to_be_clickable((By.ID, 'desp1'))).click#abre inputs nombre apellidos
        #time.sleep(1)
        nombre = i['NOMBRE'].split(' ')[0]
        a = i['NOMBRE'].split(' ')
        apellido=a[len(a)-1]
        #time.sleep(4)
        # Esperar hasta que el elemento con el ID 'nombre' sea visible
        elemento_nombre = WebDriverWait(driver_2, 5).until(EC.visibility_of_element_located((By.ID, 'nombre')))
        #elemento_nombre.clear()  # Limpiar el campo de nombre
        elemento_nombre.send_keys(nombre)

        # Esperar hasta que el elemento con el ID 'apellido1' sea visible
        elemento_apellido = WebDriverWait(driver_2, 5).until(EC.visibility_of_element_located((By.ID, 'apellido1')))
        #elemento_apellido.clear()  # Limpiar el campo de apellido
        elemento_apellido.send_keys(apellido)
        #time.sleep(2)
        WebDriverWait(driver_2, 10).until(EC.element_to_be_clickable((By.ID, 'enviar'))).click()

        # Pausa para asegurar que la página haya cargado completamente
        time.sleep(4)

        # Obtener la URL de la imagen (reemplaza el xpath con el correcto)
                                                      
        img_element = driver_2.find_element(By.XPATH, "/html/body/section[2]/div/div[2]/div/div/div[1]/a/img")
        img_data_base64 = img_element.get_attribute("src").split(",")[1]  # Extraer la parte de base64

        # Decodificar y guardar la imagen en la carpeta "imagenes"
        carpeta_imagenes = "imagenes"
        ruta_imagen = os.path.join(carpeta_imagenes, f"{i['NOMBRE']}.png")

        # Guardar la imagen en la carpeta "imagenes"
        os.makedirs(carpeta_imagenes, exist_ok=True)
        with open(ruta_imagen, "wb") as img_file:
            img_file.write(base64.b64decode(img_data_base64))

        # Añadir la ruta de la imagen al diccionario actual
        i['imagen'] = ruta_imagen

    finally:
        # Limpiar campos de nombre y apellido1 para una nueva búsqueda
        #time.sleep(2)
        driver_2.find_element(By.XPATH, "/html/body/header/div[1]/div/div[3]/div/div[2]/a").click()
        WebDriverWait(driver_2, 5).until(EC.element_to_be_clickable((By.ID, 'desp1'))).click()

        #elemento_nombre.clear()
        #elemento_apellido.clear()

# Cerrar el navegador después de realizar todas las operaciones
driver_2.quit()

# Imprimir la lista después de cerrar el navegador
print(lista_prueba)
print(lista_fallos)

Error al procesar VIKTORIIA KUDRIA
[{'NOMBRE': 'VIKTORIIA KUDRIA', 'DESAPARECE': '30 de enero de 2024', 'FECHA NACIMIENTO': '02 de octubre de 1981', 'EDAD AL DESAPARECER': '42 años', 'EDAD ACTUAL': '42 años', 'LUGAR DE DESAPARICIÓN': 'ELCHE/ELX. ALICANTE/ALACANT, COMUNIDAD VALENCIANA', 'imagen': 'imagenes\\VIKTORIIA KUDRIA.png'}]
[]


In [26]:
# Tu lista de diccionarios
desaparecidos = [
    {'NOMBRE': 'LOLA MERINO PEREZ', 'DESAPARECE': '16 de enero de 2024', 'FECHA NACIMIENTO': '19 de marzo de 2009', 'EDAD AL DESAPARECER': '14 años', 'EDAD ACTUAL': '14 años', 'LUGAR DE DESAPARICIÓN': 'JEREZ DE LA FRONTERA. CÁDIZ, ANDALUCÍA', 'imagen': 'imagenes\\LOLA MERINO PEREZ.png'}, 

{'NOMBRE': 'SELENE HEREDIA BERMUDEZ', 'DESAPARECE': '15 de enero de 2024', 'FECHA NACIMIENTO': '21 de octubre de 2006', 'EDAD AL DESAPARECER': '17 años', 'EDAD ACTUAL': '17 años', 'LUGAR DE DESAPARICIÓN': 'TOLEDO. TOLEDO, CASTILLA LA MANCHA', 'imagen': 'imagenes\\SELENE HEREDIA BERMUDEZ.png'},

{'NOMBRE': 'RAFAEL PARRA ECHAVARRIA', 'DESAPARECE': '16 de enero de 2024', 'FECHA NACIMIENTO': '14 de octubre de 1995', 'EDAD AL DESAPARECER': '28 años', 'EDAD ACTUAL': '28 años', 'LUGAR DE DESAPARICIÓN': 'MADRID. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\RAFAEL PARRA ECHAVARRIA.png'},

{'NOMBRE': 'IDELFONSO MEDRANO FERNANDEZ', 'DESAPARECE': '06 de octubre de 2021', 'FECHA NACIMIENTO': '28 de febrero de 1967', 'EDAD AL DESAPARECER': '54 años', 'EDAD ACTUAL': '56 años', 'LUGAR DE DESAPARICIÓN': 'MUTXAMEL. ALICANTE/ALACANT, COMUNIDAD VALENCIANA', 'imagen': 'imagenes\\IDELFONSO MEDRANO FERNANDEZ.png'}, 

{'NOMBRE': 'SONIA HERNANDEZ MONTES', 'DESAPARECE': '17 de diciembre de 2023', 'FECHA NACIMIENTO': '03 de mayo de 2009', 'EDAD AL DESAPARECER': '14 años', 'EDAD ACTUAL': '14 años', 'LUGAR DE DESAPARICIÓN': 'ZARAGOZA. ZARAGOZA, ARAGÓN', 'imagen': 'imagenes\\SONIA HERNANDEZ MONTES.png'}, 

{'NOMBRE': 'DANIEL OSCAR LEFLER MARZAN', 'DESAPARECE': '18 de enero de 2024', 'FECHA NACIMIENTO': '11 de mayo de 1963', 'EDAD AL DESAPARECER': '60 años', 'EDAD ACTUAL': '60 años', 'LUGAR DE DESAPARICIÓN': 'MADRID. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\DANIEL OSCAR LEFLER MARZAN.png'}, 

{'NOMBRE': 'JOSE MANUEL MUÑOZ ORTUÑO', 'DESAPARECE': '17 de enero de 2024', 'FECHA NACIMIENTO': '10 de enero de 1956', 'EDAD AL DESAPARECER': '68 años', 'EDAD ACTUAL': '68 años', 'LUGAR DE DESAPARICIÓN': 'ALCANTARILLA. MURCIA, MURCIA', 'imagen': 'imagenes\\JOSE MANUEL MUÑOZ ORTUÑO.png'}, 

{'NOMBRE': 'ABRAHAN HAG PAULETE', 'DESAPARECE': '10 de julio de 2023', 'FECHA NACIMIENTO': '23 de mayo de 2007', 'EDAD AL DESAPARECER': '16 años', 'EDAD ACTUAL': '16 años', 'LUGAR DE DESAPARICIÓN': 'TORRELODONES. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\ABRAHAN HAG PAULETE.png'}, 

{'NOMBRE': 'ANA BELEN GUERRERO FERNANDEZ', 'DESAPARECE': '21 de julio de 2020', 'FECHA NACIMIENTO': '21 de julio de 1978', 'EDAD AL DESAPARECER': '42 años', 'EDAD ACTUAL': '45 años', 'LUGAR DE DESAPARICIÓN': 'ARONA. SANTA CRUZ DE TENERIFE, CANARIAS', 'imagen': 'imagenes\\ANA BELEN GUERRERO FERNANDEZ.png'}, 

{'NOMBRE': 'IDELFONSO MEDRANO FERNANDEZ', 'DESAPARECE': '06 de octubre de 2021', 'FECHA NACIMIENTO': '28 de febrero de 1967', 'EDAD AL DESAPARECER': '54 años', 'EDAD ACTUAL': '56 años', 'LUGAR DE DESAPARICIÓN': 'MUTXAMEL. ALICANTE/ALACANT, COMUNIDAD VALENCIANA', 'imagen': 'imagenes\\IDELFONSO MEDRANO FERNANDEZ.png'},

{'NOMBRE': 'JANIS BENINS', 'DESAPARECE': '04 de abril de 2023', 'FECHA NACIMIENTO': '06 de marzo de 1967', 'EDAD AL DESAPARECER': '56 años', 'EDAD ACTUAL': '56 años', 'LUGAR DE DESAPARICIÓN': 'GERINDOTE. TOLEDO, CASTILLA LA MANCHA'},

{'NOMBRE': 'ANIS CHAABNA', 'DESAPARECE': '10 de enero de 2022', 'FECHA NACIMIENTO': '09 de agosto de 2004', 'EDAD AL DESAPARECER': '17 años', 'EDAD ACTUAL': '19 años', 'LUGAR DE DESAPARICIÓN': 'ROTOVA. VALENCIA/VALÈNCIA, COMUNIDAD VALENCIANA'},

{'NOMBRE': 'PAULA MARÍA DORESTE HERNÁNDEZ', 'DESAPARECE': '07 de enero de 2024', 'FECHA NACIMIENTO': '20 de noviembre de 2008', 'EDAD AL DESAPARECER': '15 años', 'EDAD ACTUAL': '15 años', 'LUGAR DE DESAPARICIÓN': 'SANTA CRUZ DE TENERIFE. SANTA CRUZ DE TENERIFE, CANARIAS', 'imagen': 'imagenes\\PAULA MARÍA DORESTE HERNÁNDEZ.png'},

{'NOMBRE': 'FRANCISCO JOSE TERUEL RUIZ', 'DESAPARECE': '19 de enero de 2024', 'FECHA NACIMIENTO': '27 de mayo de 1982', 'EDAD AL DESAPARECER': '41 años', 'EDAD ACTUAL': '41 años', 'LUGAR DE DESAPARICIÓN': 'MURCIA. MURCIA, MURCIA', 'imagen': 'imagenes\\FRANCISCO JOSE TERUEL RUIZ.png'}, 

{'NOMBRE': 'AINHOA AYUSO PULGAR', 'DESAPARECE': '19 de enero de 2024', 'FECHA NACIMIENTO': '20 de febrero de 2006', 'EDAD AL DESAPARECER': '17 años', 'EDAD ACTUAL': '17 años', 'LUGAR DE DESAPARICIÓN': 'PARACUELLOS DE JARAMA. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\AINHOA AYUSO PULGAR.png'},

{'NOMBRE': 'JOSE MIGUEL HERRERA GONZALEZ', 'DESAPARECE': '20 de enero de 2024', 'FECHA NACIMIENTO': '08 de septiembre de 1964', 'EDAD AL DESAPARECER': '59 años', 'EDAD ACTUAL': '59 años', 'LUGAR DE DESAPARICIÓN': 'GUIA DE ISORA. SANTA CRUZ DE TENERIFE, CANARIAS', 'imagen': 'imagenes\\JOSE MIGUEL HERRERA GONZALEZ.png'},

{'NOMBRE': 'JOSE ANGEL SEQUEIROS LOPEZ', 'DESAPARECE': '16 de enero de 2024', 'FECHA NACIMIENTO': '03 de noviembre de 2007', 'EDAD AL DESAPARECER': '16 años', 'EDAD ACTUAL': '16 años', 'LUGAR DE DESAPARICIÓN': 'POBLA DE VALLBONA (LA). VALENCIA/VALÈNCIA, COMUNIDAD VALENCIANA'},

{'NOMBRE': 'SOFIA AINHOA RIOS HUIZINGA', 'DESAPARECE': '01 de febrero de 2023', 'FECHA NACIMIENTO': '09 de abril de 2022', 'EDAD AL DESAPARECER': '9 meses', 'EDAD ACTUAL': '1 año', 'LUGAR DE DESAPARICIÓN': 'ESPAÑA', 'imagen': 'imagenes\\SOFIA AINHOA RIOS HUIZINGA.png'},

{'NOMBRE': 'ALBA RODRIGUEZ PEREZ', 'DESAPARECE': '04 de octubre de 2023', 'FECHA NACIMIENTO': '08 de mayo de 2007', 'EDAD AL DESAPARECER': '16 años', 'EDAD ACTUAL': '16 años', 'LUGAR DE DESAPARICIÓN': 'TORRELODONES. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\ALBA RODRIGUEZ PEREZ.png'},

{'NOMBRE': 'IOSIF DANIEL COLTA', 'DESAPARECE': '28 de diciembre de 2020', 'FECHA NACIMIENTO': '04 de abril de 1975', 'EDAD AL DESAPARECER': '45 años', 'EDAD ACTUAL': '48 años', 'LUGAR DE DESAPARICIÓN': 'ALEMANIA', 'imagen': 'imagenes\\IOSIF DANIEL COLTA.png'}, 

{'NOMBRE': 'FELIX JOSE ESQUERDO MARTINEZ', 'DESAPARECE': '02 de octubre de 2020', 'FECHA NACIMIENTO': '25 de febrero de 1986', 'EDAD AL DESAPARECER': '34 años', 'EDAD ACTUAL': '37 años', 'LUGAR DE DESAPARICIÓN': 'ALICANTE/ALACANT. ALICANTE/ALACANT, COMUNIDAD VALENCIANA', 'imagen': 'imagenes\\FELIX JOSE ESQUERDO MARTINEZ.png'},

{'NOMBRE': 'HENRY ALEJANDRO JIMENEZ MARIN', 'DESAPARECE': '01 de enero de 2019', 'FECHA NACIMIENTO': '18 de marzo de 1998', 'EDAD AL DESAPARECER': '20 años', 'EDAD ACTUAL': '25 años', 'LUGAR DE DESAPARICIÓN': 'ORIHUELA. ALICANTE/ALACANT, COMUNIDAD VALENCIANA', 'imagen': 'imagenes\\HENRY ALEJANDRO JIMENEZ MARIN.png'}, 

{'NOMBRE': 'YOSMAIRIS ALCANTARA MORILLO', 'DESAPARECE': '23 de enero de 2024', 'FECHA NACIMIENTO': '19 de agosto de 2008', 'EDAD AL DESAPARECER': '15 años', 'EDAD ACTUAL': '15 años', 'LUGAR DE DESAPARICIÓN': 'ZARAGOZA. ZARAGOZA, ARAGÓN', 'imagen': 'imagenes\\YOSMAIRIS ALCANTARA MORILLO.png'},

{'NOMBRE': 'RAYMON FRANCISCUS RAYMON FRANCISCUS', 'DESAPARECE': '26 de enero de 2024', 'FECHA NACIMIENTO': '23 de octubre de 2003', 'EDAD AL DESAPARECER': '20 años', 'EDAD ACTUAL': '20 años', 'LUGAR DE DESAPARICIÓN': 'MADRID. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\RAYMON FRANCISCUS RAYMON FRANCISCUS.png'},

{'NOMBRE': 'CAROLINA GARCIA MAGAN', 'DESAPARECE': '26 de enero de 2024', 'FECHA NACIMIENTO': '13 de marzo de 2007', 'EDAD AL DESAPARECER': '16 años', 'EDAD ACTUAL': '16 años', 'LUGAR DE DESAPARICIÓN': 'TRES CANTOS. MADRID, COMUNIDAD DE MADRID', 'imagen': 'imagenes\\CAROLINA GARCIA MAGAN.png'},

{'NOMBRE': 'ABRAZAK HADAK', 'DESAPARECE': '28 de enero de 2024', 'FECHA NACIMIENTO': '16 de abril de 2006', 'EDAD AL DESAPARECER': '17 años', 'EDAD ACTUAL': '17 años', 'LUGAR DE DESAPARICIÓN': 'ALBORACHE. VALENCIA/VALÈNCIA, COMUNIDAD VALENCIANA', 'imagen': 'imagenes\\ABRAZAK HADAK.png'},

{'NOMBRE': 'OSVALDO MÁRQUEZ GONZÁLEZ', 'DESAPARECE': '27 de enero de 2024', 'FECHA NACIMIENTO': '03 de enero de 1987', 'EDAD AL DESAPARECER': '37 años', 'EDAD ACTUAL': '37 años', 'LUGAR DE DESAPARICIÓN': 'PUERTO DE LA CRUZ. SANTA CRUZ DE TENERIFE, CANARIAS', 'imagen': 'imagenes\\OSVALDO MÁRQUEZ GONZÁLEZ.png'},

{'NOMBRE': 'VIKTORIIA KUDRIA', 'DESAPARECE': '30 de enero de 2024', 'FECHA NACIMIENTO': '02 de octubre de 1981', 'EDAD AL DESAPARECER': '42 años', 'EDAD ACTUAL': '42 años', 'LUGAR DE DESAPARICIÓN': 'ELCHE/ELX. ALICANTE/ALACANT, COMUNIDAD VALENCIANA', 'imagen': 'imagenes\\VIKTORIIA KUDRIA.png'}]

In [27]:
import pandas as pd


# Convertir la lista de diccionarios a un DataFrame
df = pd.DataFrame(desaparecidos)

# Guardar en un archivo CSV
df.to_csv('desaparecidos2.csv', index=False)

# Guardar en un archivo JSON
df.to_json('desaparecidos2.json', orient='records', lines=True)
